In [5]:
# Optimized YOLOv8 Pipeline for HP Elite Mini (16GB RAM)
# Complete version with all utility functions and fixes

import cv2
import json
import numpy as np
import pandas as pd
from pathlib import Path
import os
from ultralytics import YOLO
import gc
import time
import yaml
import psutil

# ====================== CONFIGURATION ======================
ELITE_MINI_CONFIG = {
    'yolo_model': 'yolov8m.pt',  # Medium model for better accuracy
    'img_size': 640,              # Input image size
    'batch_size': 4,              # Reduced for RAM safety
    'epochs': 50,                 # More epochs for better convergence
    'patience': 10,               # Early stopping patience
    'target_size': (224, 224),    # Resize for preprocessing
    'frame_skip': 2,              # Process every 2nd frame in videos
    'max_frames': 6000,           # Limit total frames to process
    'workers': 6,                 # Parallel data loading
    'cache_ram': True,            # Cache dataset in RAM
    'output_dir': 'run_yolo8m',   # Custom output folder
}

# ====================== UTILITY FUNCTIONS ======================
def monitor_memory():
    """Check system and process memory usage"""
    memory = psutil.virtual_memory()
    print(f"RAM Usage: {memory.percent}% ({memory.used/1024**3:.1f}GB/{memory.total/1024**3:.1f}GB)")
    process = psutil.Process()
    print(f"Process Memory: {process.memory_info().rss/1024**2:.1f}MB")

def load_annotated_data_info(data_root):
    """Load annotations from CSV/JSON or video files (FIXED SYNTAX)"""
    data_root = Path(data_root)
    if not data_root.exists():
        raise FileNotFoundError(f"Data directory not found: {data_root}")
    
    annotation_files = list(data_root.glob("*.csv")) + list(data_root.glob("*.json"))
    dfs = []
    
    if annotation_files:
        for file in annotation_files:
            try:
                if file.suffix == '.csv':
                    dfs.append(pd.read_csv(file))
                else:  # JSON file
                    with open(file, 'r') as f:
                        data = json.load(f)
                    if isinstance(data, list):
                        dfs.append(pd.DataFrame(data))
                    else:
                        dfs.append(pd.DataFrame([data]))  # Wrap dict in list
            except Exception as e:
                print(f"Error loading {file.name}: {e}")
        return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()
    
    # Fallback to video files if no annotations found
    video_files = list(data_root.glob("*.mp4")) + list(data_root.glob("*.avi"))
    if video_files:
        return create_dummy_annotations(video_files)
    return pd.DataFrame()

def create_dummy_annotations(video_files, frames_step=30):
    """Create placeholder annotations if no real data exists"""
    annotations = []
    for video in video_files[:5]:  # Use first 5 videos max
        cap = cv2.VideoCapture(str(video))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        cap.release()
        
        for frame_num in range(0, min(total_frames, 300), frames_step):
            annotations.append({
                'video_path': str(video),
                'video_id': video.stem,
                'frame_num': frame_num,
                'bbox': [100, 100, 200, 200],  # Dummy bounding box
                'behavior': 'pig',
                'behavior_encoded': 0
            })
    return pd.DataFrame(annotations)

# ====================== CORE CLASSES ======================
class EliteMiniYOLODetector:
    def __init__(self, config=ELITE_MINI_CONFIG):
        self.config = config
        self.model = YOLO(config['yolo_model'])
    
    def train(self, data_yaml):
        """Optimized training for 16GB RAM"""
        results = self.model.train(
            data=data_yaml,
            epochs=self.config['epochs'],
            imgsz=self.config['img_size'],
            batch=self.config['batch_size'],
            project=self.config['output_dir'],
            name='train',
            cache=self.config['cache_ram'],
            device='cpu',
            workers=self.config['workers']
        )
        return results
    
    def detect(self, frame):
        """Optimized inference"""
        results = self.model.predict(frame, verbose=False)
        return results[0].boxes.data.cpu().numpy()

# ====================== DATASET PREPARATION ======================
def prepare_dataset(data_info, config=ELITE_MINI_CONFIG):
    """
    Creates 'elite_mini_dataset' folder because:
    1. YOLO requires specific directory structure (images/, labels/)
    2. Converts annotations to YOLO format (normalized coordinates)
    3. Centralizes all training data in one place
    """
    dataset_dir = Path(config['output_dir']) / 'dataset'
    (dataset_dir / 'images').mkdir(parents=True, exist_ok=True)
    (dataset_dir / 'labels').mkdir(parents=True, exist_ok=True)
    
    # Create data.yaml configuration
    yaml_data = {
        'path': str(dataset_dir.absolute()),
        'train': 'images',
        'val': 'images',  # Using same set for simplicity
        'names': ['pig'],
        'nc': 1
    }
    with open(dataset_dir / 'data.yaml', 'w') as f:
        yaml.dump(yaml_data, f)
    
    # Process and save frames
    for idx, row in data_info.iterrows():
        try:
            # Load and resize frame
            cap = cv2.VideoCapture(row['video_path'])
            cap.set(cv2.CAP_PROP_POS_FRAMES, row['frame_num'])
            ret, frame = cap.read()
            cap.release()
            
            if not ret:
                continue
                
            frame = cv2.resize(frame, (config['img_size'], config['img_size']))
            img_path = dataset_dir / 'images' / f"{row['video_id']}_{row['frame_num']}.jpg"
            cv2.imwrite(str(img_path), frame)
            
            # Convert bbox to YOLO format
            bbox = eval(row['bbox']) if isinstance(row['bbox'], str) else row['bbox']
            x_center = (bbox[0] + bbox[2]/2) / config['img_size']
            y_center = (bbox[1] + bbox[3]/2) / config['img_size']
            width = bbox[2] / config['img_size']
            height = bbox[3] / config['img_size']
            
            # Save label
            label_path = dataset_dir / 'labels' / f"{row['video_id']}_{row['frame_num']}.txt"
            with open(label_path, 'w') as f:
                f.write(f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
                
        except Exception as e:
            print(f"Error processing frame {idx}: {e}")

# ====================== MAIN PIPELINE ======================
def run_pipeline(data_root):
    """End-to-end training pipeline"""
    print("=== Memory Check ===")
    monitor_memory()
    
    print("\n=== Loading Data ===")
    data_info = load_annotated_data_info(data_root)
    if data_info.empty:
        raise ValueError("No valid data found!")
    
    print("\n=== Preparing Dataset ===")
    prepare_dataset(data_info)
    
    print("\n=== Training Model ===")
    detector = EliteMiniYOLODetector()
    detector.train(str(Path(ELITE_MINI_CONFIG['output_dir']) / 'dataset/data.yaml'))
    
    print("\n=== Final Memory Check ===")
    monitor_memory()
    return detector

if __name__ == "__main__":
    # Configure your paths here
    DATA_ROOT = "C:/Users/2955352g/Desktop/pig_data_edinburgh"
    TEST_VIDEO = "test_video.mp4"
    
    # Run the pipeline
    trained_detector = run_pipeline(DATA_ROOT)
    
    # Optional: Test on a video
    if os.path.exists(TEST_VIDEO):
        print("\n=== Testing on Video ===")
        cap = cv2.VideoCapture(TEST_VIDEO)
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            detections = trained_detector.detect(frame)
            print(f"Detected {len(detections)} objects")
        cap.release()

=== Memory Check ===
RAM Usage: 63.5% (10.0GB/15.7GB)
Process Memory: 275.0MB

=== Loading Data ===


ValueError: No valid data found!

In [8]:
# Optimized Multi-Pig Detection for HP Mini with YOLOv8m
import cv2
import json
import numpy as np
import pandas as pd
from pathlib import Path
import os
from ultralytics import YOLO
import gc
import time

# Updated configurations for HP Elite Mini 800 G9 (i5-12500T) with 16GB RAM
ELITE_MINI_CONFIG = {
    'yolo_model': 'yolov8m.pt',  # Medium model for better detection
    'img_size': 640,  # Standard size
    'batch_size': 4,  # Reduced batch size for medium model
    'epochs': 30,  # More epochs with sufficient RAM
    'patience': 7,  # Longer patience
    'target_size': (224, 224),  # Standard target size
    'frame_skip': 3,  # Process more frames
    'max_frames': 6000,  # Slightly reduced due to larger model
    'workers': 6,  # Fewer workers to balance memory
    'cache_ram': True,  # Enable RAM caching
    'pig_colors': [(0, 255, 0), (0, 0, 255), (255, 0, 0), (255, 255, 0), (255, 0, 255)],  # Different colors for pigs
}

class OptimizedDataGenerator:
    """Memory-efficient data generator for HP Mini"""
    
    def __init__(self, data_info, config=ELITE_MINI_CONFIG):
        self.data_info = data_info[:config['max_frames']]  # Limit data
        self.config = config
        self.current_idx = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.current_idx >= len(self.data_info):
            raise StopIteration
            
        data = self.data_info.iloc[self.current_idx]
        self.current_idx += 1
        
        try:
            # Load and process frame
            cap = cv2.VideoCapture(str(data['video_path']))
            cap.set(cv2.CAP_PROP_POS_FRAMES, data['frame_num'])
            ret, frame = cap.read()
            cap.release()
            
            if ret:
                # Resize to smaller size
                frame = cv2.resize(frame, self.config['target_size'])
                frame = frame.astype(np.float32) / 255.0
                return frame, data['behavior_encoded']
            else:
                return self.__next__()  # Skip failed frames
                
        except Exception as e:
            print(f"Error loading frame: {e}")
            return self.__next__()

class EliteMiniYOLODetector:
    """Optimized YOLO detector for HP Elite Mini 800 G9 with multi-pig detection"""
    
    def __init__(self, config=ELITE_MINI_CONFIG):
        self.config = config
        self.model = YOLO(config['yolo_model'])
        
    def train_optimized(self, data_yaml):
        """Train with Elite Mini optimizations"""
        print("Training with Elite Mini 800 G9 optimizations (16GB RAM)...")
        
        results = self.model.train(
            data=data_yaml,
            epochs=self.config['epochs'],
            imgsz=self.config['img_size'],
            batch=self.config['batch_size'],
            device='cpu',
            workers=self.config['workers'],
            patience=self.config['patience'],
            cache=self.config['cache_ram'],  # Enable RAM caching
            save_period=10,
            verbose=True,
            amp=False,  # Keep disabled for CPU
            optimizer='AdamW',
            lr0=0.01,
            weight_decay=0.0005,
            momentum=0.937,
        )
        return results
    
    def detect_optimized(self, frame, conf=0.3):
        """Optimized detection for HP Mini with multiple pig detection and colored bboxes"""
        # Resize frame to smaller size
        small_frame = cv2.resize(frame, (self.config['img_size'], self.config['img_size']))
        
        # Run inference
        results = self.model.predict(
            small_frame,
            conf=conf,
            device='cpu',
            verbose=False,
            half=False,  # Disable half precision
        )
        
        # Scale detections back to original size
        h, w = frame.shape[:2]
        scale_x = w / self.config['img_size']
        scale_y = h / self.config['img_size']
        
        detections = []
        color_index = 0
        
        for result in results:
            if result.boxes is not None:
                boxes = result.boxes.xyxy.cpu().numpy()
                confs = result.boxes.conf.cpu().numpy()
                
                for box, conf in zip(boxes, confs):
                    # Scale back to original size
                    x1, y1, x2, y2 = box
                    x1, x2 = int(x1 * scale_x), int(x2 * scale_x)
                    y1, y2 = int(y1 * scale_y), int(y2 * scale_y)
                    
                    # Get color for this pig (cycle through colors)
                    color = self.config['pig_colors'][color_index % len(self.config['pig_colors'])]
                    color_index += 1
                    
                    detections.append({
                        'bbox': [x1, y1, x2-x1, y2-y1],
                        'confidence': float(conf),
                        'color': color  # Store color with detection
                    })
        
        return detections

def prepare_standard_dataset(data_info, output_dir, config=ELITE_MINI_CONFIG):
    """Prepare standard dataset for Elite Mini with multiple pig support"""
    output_dir = Path(output_dir)
    (output_dir / 'images').mkdir(parents=True, exist_ok=True)
    (output_dir / 'labels').mkdir(parents=True, exist_ok=True)
    
    # Can handle more frames
    limited_data = data_info.head(config['max_frames'])
    
    # Create data.yaml
    data_yaml = {
        'path': str(output_dir.absolute()),
        'train': 'images',
        'val': 'images',
        'names': ['pig'],
        'nc': 1
    }
    
    import yaml
    with open(output_dir / 'data.yaml', 'w') as f:
        yaml.dump(data_yaml, f)
    
    print(f"Processing {len(limited_data)} frames...")
    
    for idx, row in limited_data.iterrows():
        if idx % 200 == 0:
            print(f"Processed {idx}/{len(limited_data)} frames")
            gc.collect()
        
        try:
            # Load frame
            cap = cv2.VideoCapture(str(row['video_path']))
            cap.set(cv2.CAP_PROP_POS_FRAMES, row['frame_num'])
            ret, frame = cap.read()
            cap.release()
            
            if not ret:
                continue
            
            # Keep original size for better quality
            frame = cv2.resize(frame, (config['img_size'], config['img_size']))
            
            # Save image with good quality
            img_path = output_dir / 'images' / f"{row['video_id']}_{row['frame_num']}.jpg"
            cv2.imwrite(str(img_path), frame, [cv2.IMWRITE_JPEG_QUALITY, 95])
            
            # Process bbox (now handling multiple pigs per frame)
            bboxes = row['bbox']
            if isinstance(bboxes, str):
                import ast
                bboxes = ast.literal_eval(bboxes)
            
            # Convert single bbox to list format
            if isinstance(bboxes, dict):
                bboxes = [bboxes]
            
            # Prepare label file content
            label_content = []
            img_h, img_w = config['img_size'], config['img_size']
            
            for bbox in bboxes:
                if isinstance(bbox, dict):
                    x = bbox.get('x', 0)
                    y = bbox.get('y', 0)
                    w = bbox.get('width', bbox.get('w', 100))
                    h = bbox.get('height', bbox.get('h', 100))
                else:
                    x, y, w, h = bbox
                
                # Convert to YOLO format
                x_center = (x + w/2) / img_w
                y_center = (y + h/2) / img_h
                width = w / img_w
                height = h / img_h
                
                # Clamp values
                x_center = max(0, min(1, x_center))
                y_center = max(0, min(1, y_center))
                width = max(0, min(1, width))
                height = max(0, min(1, height))
                
                label_content.append(f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
            
            # Save label
            label_path = output_dir / 'labels' / f"{row['video_id']}_{row['frame_num']}.txt"
            with open(label_path, 'w') as f:
                f.write('\n'.join(label_content))
                
        except Exception as e:
            print(f"Error processing frame {idx}: {e}")
            continue

def load_annotated_data_info(data_root="C:/Users/2955352g/Desktop/pig_data_edinburgh"):
    """Load annotated data from various possible formats"""
    data_root = Path(data_root)
    
    print(f"Searching for data in: {data_root}")
    
    # Check what files exist in the data directory
    if not data_root.exists():
        print(f"Data directory {data_root} does not exist!")
        return pd.DataFrame()
    
    # List all files in the directory
    print("Files found in data directory:")
    for item in data_root.rglob("*"):
        if item.is_file():
            print(f"  {item.name} ({item.suffix})")
    
    # Try to load from common annotation formats
    annotation_data = []
    
    # Method 1: Try to load from CSV files
    csv_files = list(data_root.glob("*.csv"))
    if csv_files:
        print(f"\nFound {len(csv_files)} CSV files:")
        for csv_file in csv_files:
            print(f"  {csv_file.name}")
            try:
                df = pd.read_csv(csv_file)
                print(f"    Columns: {list(df.columns)}")
                print(f"    Shape: {df.shape}")
                annotation_data.append(df)
            except Exception as e:
                print(f"    Error reading {csv_file}: {e}")
    
    # Method 2: Try to load from JSON files
    json_files = list(data_root.glob("*.json"))
    if json_files:
        print(f"\nFound {len(json_files)} JSON files:")
        for json_file in json_files:
            print(f"  {json_file.name}")
            try:
                with open(json_file, 'r') as f:
                    data = json.load(f)
                print(f"    Keys: {list(data.keys()) if isinstance(data, dict) else 'List format'}")
                # Convert JSON to DataFrame format if needed
                if isinstance(data, list):
                    df = pd.DataFrame(data)
                    annotation_data.append(df)
                elif isinstance(data, dict):
                    df = pd.DataFrame([data])
                    annotation_data.append(df)
            except Exception as e:
                print(f"    Error reading {json_file}: {e}")
    
    # Method 3: Look for video files and create basic structure
    video_extensions = ['.mp4', '.avi', '.mov', '.mkv']
    video_files = []
    for ext in video_extensions:
        video_files.extend(data_root.glob(f"*{ext}"))
        video_files.extend(data_root.glob(f"**/*{ext}"))
    
    if video_files:
        print(f"\nFound {len(video_files)} video files:")
        for video_file in video_files[:5]:  # Show first 5
            print(f"  {video_file.name}")
        if len(video_files) > 5:
            print(f"  ... and {len(video_files) - 5} more")
    
    # If we found annotation data, try to standardize it
    if annotation_data:
        print(f"\nProcessing {len(annotation_data)} annotation files...")
        
        # Combine all annotation data
        combined_df = pd.concat(annotation_data, ignore_index=True)
        
        # Try to standardize column names
        standardized_df = standardize_annotation_format(combined_df, data_root)
        
        if not standardized_df.empty:
            print(f"Successfully loaded {len(standardized_df)} annotations")
            return standardized_df
    
    # If no annotation files found, create a basic structure from video files
    if video_files:
        print("\nNo annotation files found. Creating basic structure from video files...")
        return create_basic_annotations_from_videos(video_files)
    
    print("\nNo suitable data found!")
    return pd.DataFrame()

def standardize_annotation_format(df, data_root):
    """Try to standardize annotation format to expected columns"""
    print("Attempting to standardize annotation format...")
    print(f"Input columns: {list(df.columns)}")
    
    # Common column name mappings
    column_mappings = {
        # Video path variations
        'video_path': ['video_path', 'video_file', 'video', 'filename', 'file_path'],
        'video_id': ['video_id', 'video_name', 'id', 'video', 'file_id'],
        
        # Frame number variations
        'frame_num': ['frame_num', 'frame_number', 'frame', 'frame_id', 'timestamp'],
        
        # Bounding box variations
        'bbox': ['bbox', 'bounding_box', 'box', 'coordinates', 'bounds'],
        'x': ['x', 'x1', 'left', 'bbox_x'],
        'y': ['y', 'y1', 'top', 'bbox_y'],
        'width': ['width', 'w', 'bbox_width', 'bbox_w'],
        'height': ['height', 'h', 'bbox_height', 'bbox_h'],
        
        # Behavior/label variations
        'behavior': ['behavior', 'label', 'class', 'category', 'behavior_label'],
        'behavior_encoded': ['behavior_encoded', 'label_encoded', 'class_id'],
    }
    
    # Create new standardized dataframe
    standardized_data = []
    
    for idx, row in df.iterrows():
        try:
            # Extract video information
            video_path = None
            video_id = None
            
            for col in df.columns:
                if any(vid_col in col.lower() for vid_col in column_mappings['video_path']):
                    video_path = row[col]
                    break
            
            for col in df.columns:
                if any(vid_col in col.lower() for vid_col in column_mappings['video_id']):
                    video_id = row[col]
                    break
            
            # If video_path is relative, make it absolute
            if video_path and not os.path.isabs(video_path):
                video_path = data_root / video_path
            
            # Extract frame number
            frame_num = 0
            for col in df.columns:
                if any(frame_col in col.lower() for frame_col in column_mappings['frame_num']):
                    frame_num = row[col]
                    break
            
            # Extract bounding box
            bbox = None
            
            # Try to find bbox as single column
            for col in df.columns:
                if any(bbox_col in col.lower() for bbox_col in column_mappings['bbox']):
                    bbox = row[col]
                    break
            
            # If no single bbox column, try to construct from x,y,w,h
            if bbox is None:
                x, y, w, h = None, None, None, None
                
                for col in df.columns:
                    col_lower = col.lower()
                    if any(x_col in col_lower for x_col in column_mappings['x']):
                        x = row[col]
                    elif any(y_col in col_lower for y_col in column_mappings['y']):
                        y = row[col]
                    elif any(w_col in col_lower for w_col in column_mappings['width']):
                        w = row[col]
                    elif any(h_col in col_lower for h_col in column_mappings['height']):
                        h = row[col]
                
                if all(v is not None for v in [x, y, w, h]):
                    bbox = [x, y, w, h]
            
            # Extract behavior
            behavior = None
            behavior_encoded = 0
            
            for col in df.columns:
                if any(beh_col in col.lower() for beh_col in column_mappings['behavior']):
                    behavior = row[col]
                    break
            
            for col in df.columns:
                if any(beh_col in col.lower() for beh_col in column_mappings['behavior_encoded']):
                    behavior_encoded = row[col]
                    break
            
            # Create standardized entry
            if video_path and bbox:
                standardized_data.append({
                    'video_path': str(video_path),
                    'video_id': video_id or f"video_{idx}",
                    'frame_num': int(frame_num),
                    'bbox': bbox,
                    'behavior': behavior or "pig_behavior",
                    'behavior_encoded': int(behavior_encoded)
                })
                
        except Exception as e:
            print(f"Error processing row {idx}: {e}")
            continue
    
    if standardized_data:
        result_df = pd.DataFrame(standardized_data)
        print(f"Successfully standardized {len(result_df)} annotations")
        return result_df
    else:
        print("Could not standardize any annotations")
        return pd.DataFrame()

def create_basic_annotations_from_videos(video_files):
    """Create basic annotations from video files (for testing)"""
    print("Creating basic annotations from video files...")
    
    annotations = []
    
    for video_file in video_files[:10]:  # Limit to first 10 videos
        try:
            # Get video info
            cap = cv2.VideoCapture(str(video_file))
            if not cap.isOpened():
                continue
                
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            cap.release()
            
            # Create annotations for every 30th frame (sample)
            for frame_num in range(0, min(total_frames, 3000), 30):
                annotations.append({
                    'video_path': str(video_file),
                    'video_id': video_file.stem,
                    'frame_num': frame_num,
                    'bbox': [100, 100, 200, 200],  # Dummy bbox - you'll need real annotations
                    'behavior': 'pig_behavior',
                    'behavior_encoded': 0
                })
        except Exception as e:
            print(f"Error processing video {video_file}: {e}")
            continue
    
    if annotations:
        print(f"Created {len(annotations)} basic annotations")
        print("⚠️  WARNING: Using dummy bounding boxes! You need real annotations for proper training.")
        return pd.DataFrame(annotations)
    else:
        return pd.DataFrame()

def elite_mini_pipeline(data_root):
    """Optimized pipeline for HP Elite Mini 800 G9 with multi-pig detection"""
    print("Starting Elite Mini 800 G9 optimized pipeline with multi-pig detection...")
    
    # Load data
    print("Loading data information...")
    data_info = load_annotated_data_info(data_root)
    
    if data_info.empty:
        print("No data found - please implement load_annotated_data_info()")
        return None
    
    # Can handle more data
    data_info = data_info.head(ELITE_MINI_CONFIG['max_frames'])
    print(f"Using {len(data_info)} frames for training")
    
    # Prepare dataset
    print("Preparing dataset...")
    prepare_standard_dataset(data_info, 'elite_mini_dataset')
    
    # Initialize detector
    detector = EliteMiniYOLODetector()
    
    # Train with optimizations
    print("Training detector (estimated 2-4 hours)...")
    start_time = time.time()
    
    results = detector.train_optimized('elite_mini_dataset/data.yaml')
    
    training_time = time.time() - start_time
    print(f"Training completed in {training_time/3600:.2f} hours")
    
    return detector

def test_elite_mini_video(detector, video_path):
    """Test video processing on Elite Mini with colored boxes"""
    if not os.path.exists(video_path):
        print(f"Video file {video_path} does not exist")
        return
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Cannot open video {video_path}")
        return
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Processing video with {total_frames} frames")
    
    frame_count = 0
    processed_count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        
        # Process every 5th frame (reasonable for 12-thread CPU)
        if frame_count % ELITE_MINI_CONFIG['frame_skip'] == 0:
            start_time = time.time()
            
            detections = detector.detect_optimized(frame)
            
            process_time = time.time() - start_time
            processed_count += 1
            
            # Draw detections with different colors
            for det in detections:
                x, y, w, h = det['bbox']
                color = det.get('color', (0, 255, 0))  # Default to green if no color
                cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
                cv2.putText(frame, f"Pig {det['confidence']:.2f}", (x, y-10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
            
            # Show frame
            cv2.imshow('Pig Detection', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            
            print(f"Frame {frame_count}: {len(detections)} detections, "
                  f"Processing time: {process_time:.2f}s")
            
            # Less frequent memory cleanup
            if processed_count % 100 == 0:
                gc.collect()
    
    cap.release()
    cv2.destroyAllWindows()
    print(f"Processed {processed_count} frames out of {total_frames}")
    print(f"Average processing speed: {processed_count/total_frames*100:.1f}% of frames")

def monitor_memory():
    """Monitor detailed memory usage"""
    import psutil
    
    # Overall system memory
    memory = psutil.virtual_memory()
    print(f"Total RAM: {memory.total / 1024**3:.1f} GB")
    print(f"Available RAM: {memory.available / 1024**3:.1f} GB")
    print(f"Used RAM: {memory.used / 1024**3:.1f} GB ({memory.percent:.1f}%)")
    
    # Process-specific memory
    process = psutil.Process()
    process_memory = process.memory_info()
    print(f"Process RSS: {process_memory.rss / 1024**3:.2f} GB")
    print(f"Process VMS: {process_memory.vms / 1024**3:.2f} GB")
    
    # Check if we're approaching memory limits
    if memory.percent > 85:
        print("⚠️  WARNING: High memory usage detected!")
        print("Consider reducing batch_size or max_frames")
    elif memory.percent > 70:
        print("ℹ️  Memory usage is getting high, monitoring recommended")
    else:
        print("✅ Memory usage is healthy")

def optimize_for_16gb():
    """Suggest optimizations based on current memory usage"""
    import psutil
    memory = psutil.virtual_memory()
    
    if memory.available < 4 * 1024**3:  # Less than 4GB available
        print("🔧 Reducing batch size for low available memory")
        ELITE_MINI_CONFIG['batch_size'] = 2
        ELITE_MINI_CONFIG['max_frames'] = 4000
        ELITE_MINI_CONFIG['workers'] = 4
    elif memory.available < 6 * 1024**3:  # Less than 6GB available
        print("🔧 Using moderate settings for medium available memory")
        ELITE_MINI_CONFIG['batch_size'] = 4
        ELITE_MINI_CONFIG['max_frames'] = 6000
        ELITE_MINI_CONFIG['workers'] = 6
    else:
        print("✅ Using full settings for high available memory")
        # Use default settings from config

if __name__ == "__main__":
    # Monitor initial memory and optimize settings
    print("=== Initial System Status ===")
    monitor_memory()
    optimize_for_16gb()
    
    print("\n=== Starting Training Pipeline ===")
    
    # Run Elite Mini optimized pipeline
    detector = elite_mini_pipeline("C:/Users/2955352g/Desktop/pig_data_edinburgh")
    
    if detector:
        print("Training completed successfully!")
        
        # Monitor memory after training
        print("\n=== Post-Training Memory Status ===")
        monitor_memory()
        
        # Test on video
        test_video = "test_video.mp4"  # Adjust path
        if os.path.exists(test_video):
            test_elite_mini_video(detector, test_video)
    
    # Final memory check
    print("\n=== Final Memory Status ===")
    monitor_memory()

=== Initial System Status ===
Total RAM: 15.7 GB
Available RAM: 6.6 GB
Used RAM: 9.1 GB (58.0%)
Process RSS: 0.27 GB
Process VMS: 0.61 GB
✅ Memory usage is healthy
✅ Using full settings for high available memory

=== Starting Training Pipeline ===
Starting Elite Mini 800 G9 optimized pipeline with multi-pig detection...
Loading data information...
Searching for data in: C:\Users\2955352g\Desktop\pig_data_edinburgh
Files found in data directory:
  test_video.mp4 (.mp4)
  background.png (.png)
  background_depth.png (.png)
  color.mp4 (.mp4)
  depth.mp4 (.mp4)
  depth_scale.npy (.npy)
  inverse_intrinsic.npy (.npy)
  mask.png (.png)
  output.json (.json)
  rot.npy (.npy)
  times.txt (.txt)
  background.png (.png)
  background_depth.png (.png)
  color.mp4 (.mp4)
  depth.mp4 (.mp4)
  depth_scale.npy (.npy)
  inverse_intrinsic.npy (.npy)
  mask.png (.png)
  output.json (.json)
  rot.npy (.npy)
  times.txt (.txt)
  background.png (.png)
  background_depth.png (.png)
  color.mp4 (.mp4)
  dept

100%|██████████| 49.7M/49.7M [00:51<00:00, 1.02MB/s]


Training detector (estimated 2-4 hours)...
Training with Elite Mini 800 G9 optimizations (16GB RAM)...
Ultralytics 8.3.166  Python-3.11.13 torch-2.5.1 CPU (12th Gen Intel Core(TM) i5-12500T)
engine\trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=elite_mini_dataset/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=

train: Scanning C:\Users\2955352g\Desktop\Thesis\Codes\elite_mini_dataset\labels.cache... 180 images, 0 backgrounds, 0 corrupt: 100%|██████████| 180/180 [00:00<?, ?it/s]


WARNING cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


...

      30/30         0G      1.283      2.111      1.974          4        640: 100%|██████████| 45/45 [03:15<00:00,  4.34s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [01:14<00:00,  3.25s/it]

                   all        180        180      0.648      0.383      0.523      0.379



30 epochs completed in 2.298 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 52.0MB
Optimizer stripped from runs\detect\train\weights\best.pt, 52.0MB

Validating runs\detect\train\weights\best.pt...
Ultralytics 8.3.166  Python-3.11.13 torch-2.5.1 CPU (12th Gen Intel Core(TM) i5-12500T)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [01:04<00:00,  2.80s/it]


                   all        180        180      0.698       0.41      0.535      0.381
Speed: 0.8ms preprocess, 353.3ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs\detect\train
Training completed in 2.32 hours
Training completed successfully!

=== Post-Training Memory Status ===
Total RAM: 15.7 GB
Available RAM: 5.1 GB
Used RAM: 10.5 GB (67.2%)
Process RSS: 2.38 GB
Process VMS: 3.67 GB
✅ Memory usage is healthy

=== Final Memory Status ===
Total RAM: 15.7 GB
Available RAM: 5.1 GB
Used RAM: 10.5 GB (67.2%)
Process RSS: 2.38 GB
Process VMS: 3.67 GB
✅ Memory usage is healthy
